## Project Plan

### Goals

### Hypothesis

### Components

## Acquire and Prep

### Pull SQL data and filter data frame

Using the `get_sql_zillow` function from the `acquire` module, pull all the columns from the Zillow dataset from SQL following the conditions below:
 - only include properties that have recorded transactions in 2017
 - exclude properties that do not have latitude and longitude values (location is important in our analysis later so we wante to make sure we are including observations with known locations.)

In [1]:
# get_sql_zillow code here

Using our `wrangle_zillow`, perform initial cleaning of the original data frame specifically:
 - Drop ID columns from Left Joins (type construction, storytype, etc.)
 - Remove duplicates from multiple transactions by maintaining latest transaction date and dropping previous records
 - Include property land use type "Single Family Residential" only, and drop the rest of the observations
 - Drop properties that have unit counts of 2(27 obs) 3(1 obs) to remove ambiguity in the definition of "single-unit" or "single-family" houses, especially that the total number of these observations are relatively minimal.
 - Keep transactions that are recorded in 2017 (there was 1 obs that has 2018 as transaction year)

In [2]:
# wrangle_zillow code here

### Handling missing values

`handle_missing_values` function from the `prep` module drops columns that are 90% empty, reducing our columns from 60 to 27. Then, remove observations that are 40% unfilled. The latter removed no observations.  
  
In this stage, our working data has 52292 rows with 27 columns

#### Columns to be dropped

`calculatedbathnbr` or calcualted bath number records the number of bathrooms in a property including half bathrooms. Therefore, it's data type is a float (.5's) while	`fullbathcnt` column only includes full bathrooms and discounts half baths. So, we are discarding `fullbathcnt` completely and take `calculatedbathnbr` which provides more information on the properties' bathrooms.

`calculatedfinishedsquarefeet` and `finishedsquarefeet12` hold practically the same information. `calculatedfinishedsquarefeet` has less nulls (81 obs) than `finishedsquarefeet12` (245 obs), so we are dropping the `finishedsquarefeet12`.

`propertycountylandusecode` are codes used in the industry to specify the land use. For example, a Single Family Residential property land use type may be Single Family Class II (0102), Vacant Residential (0000), Vacant Lake View (0035), etc. We don't need these further information. All we need to know is that the properties that we are looking at are under the umbrella of "Single Family Residential."

`rawcensustractandblock` and `censustractandblock` contain census information that we are not concerned about in this project.

`fips` and `regionidcounty` have cotain the same information. We will keep the `fips` column.

`regionidzip` is a location-based column which does not give added information on the properties' location given that we have `latitude` and `longitude`. `regionidzip` is dropped.

HAVE

MONEY - SQFT  
Structure TVDC - Calculated finished SQFT  
Land TVDC - (Lot size SQFT - Calculated finished SQFT)